<a href="https://colab.research.google.com/github/SergiodotICO/edd_1358_2021/blob/master/RedNeuronal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install numpy

In [2]:
import random
import math

class NN:
    def __init__(self,numInputs,numHidden,numOutputs,
                 initWeightSD,learningRate):

        ## we add 1 to the number of inputs here for the bias node
        numInputs += 1

        self.inputToHidden = [] 
        self.hiddenToOutput = []
        self.numInputs = numInputs 
        self.numHidden = numHidden
        self.numOutputs = numOutputs
        self.learningRate = learningRate

        for i in range(numInputs):
            workingList = []
            for h in range(numHidden):
                newWeight = random.normalvariate(0.0, initWeightSD)
                workingList.append(newWeight)
            self.inputToHidden.append(workingList)


        for h in range(numHidden + 1):     ## +1 to cover the bias-to-output connections
            workingList = []
            for o in range(numOutputs):
                newWeight = random.normalvariate(0.0, initWeightSD)
                workingList.append(newWeight)
            self.hiddenToOutput.append(workingList)


    def printNetwork(self, name):
        print (name, "network weights")
        for i in range(self.numInputs):
            for j in range(self.numHidden):
                print (self.inputToHidden[i][j]),
            print()
        print ("--------------------")
        for j in range(self.numHidden):
            for k in range(self.numOutputs):
                print (self.hiddenToOutput[j][k]),
            print ()


    def calcNetOutput(self,inputToNet,wantHiddenLevels=False):
        """Input is list of real-valued inputs.  Also a Boolean specifying
        whether you'd like the hidden layer activation levels returned or not.
        Output is a list of real-valued outputs, and optionally the hidden layer
        activation levels too.  Both are sigmoid-functioned before being returned.
        Note that if you get confused about whether you want the hidden layer
        levels or not, things can get messed up, as you'll get two lists returned
        instead of one."""

        ## immediately add 1.0 to the input list: this is the bias node
        inputWithBias = inputToNet[:]
        inputWithBias.append(1.0)

        hiddenActivationLevels = [ 0.0 ] * self.numHidden
        outputActivationLevels = [ 0.0 ] * self.numOutputs

        for i in range(self.numInputs):
            for h in range(self.numHidden):
                hiddenActivationLevels[h] += ( inputWithBias[i]
                                               * self.inputToHidden[i][h] )

        for h in range(self.numHidden):
            hiddenActivationLevels[h] = self.sigmoid(hiddenActivationLevels[h])

        hiddenActivationLevels.append(1.0)  ## this is the bias-to-output node
                
        for h in range(self.numHidden + 1):    ## +1 to cover the bias-to-output connections
            for o in range(self.numOutputs):
                outputActivationLevels[o] += ( hiddenActivationLevels[h]
                                               * self.hiddenToOutput[h][o] )

        ## Note that we sigmoid the output functions before we send them back
        ## This is because they need to be in the 0--1 range so we can compare
        ## them to bit strings and judge their closeness.
        for o in range(self.numOutputs):
            outputActivationLevels[o] = self.sigmoid(outputActivationLevels[o])

        if wantHiddenLevels:
            return outputActivationLevels, hiddenActivationLevels
        else:
            return outputActivationLevels


    def sigmoid(self,before):
        """Returns the classic neural-net sigmoid of the `before' value.
        So the return value is bounded by 0 and 1, but the input value
        can be anything (pos or neg, bounded by infiinity)."""
        retValue = 1.0 / (1.0 + math.exp(0.0 - before) )
        return retValue
    
    def trainingEpisode(self,targetOutput,actualOutput,
                        hiddenOutput,actualInput):
        """This is where the backpropagation maths get implemented.
        We do one training event where the weights of the network
        get properly updated for a given target output and a given
        actual output.  Both of these are lists of real values."""

        # as before, add 1.0 to the input list to implement the bias node
        inputWithBias = actualInput[:]
        inputWithBias.append(1.0)

        hiddenWithBias = hiddenOutput[:]
        hiddenWithBias.append(1.0)
        
        # calculate deltaKs: basically an error measure per output neuron
        deltaK = [ (t - y) * y * (1 - y)
                   for t, y in zip(targetOutput, actualOutput) ]

        #print "deltaK = ", deltaK

        # train the hidden-to-output connections
        for j in range(self.numHidden + 1):   ## +1 for the bias-to-output connections
            #print "HO", j, "=", hiddenWithBias[j]
            for k in range(self.numOutputs):
                deltaW = self.learningRate * deltaK[k] * hiddenWithBias[j]
                self.hiddenToOutput[j][k] += deltaW
                #print "H->O weight delta", j, k, deltaW 



        # calculate deltaJs: basically an error measure for hidden neurons
        deltaJ = [ 0.0 ] * self.numHidden
        for j in range(self.numHidden):  ## deliberate use of numHidden, not numHidden + 1
            sigma = 0.0
            for k in range(self.numOutputs):
                sigma += self.hiddenToOutput[j][k] * deltaK[k]
            deltaJ[j] = hiddenOutput[j] * ( 1 - hiddenOutput[j] ) * sigma

        #print "deltaJ = ", deltaJ

        # train the input-to-hidden connections
        for i in range(self.numInputs):
            #print "AI", i, "=", inputWithBias[i]
            for j in range(self.numHidden):
                deltaW = self.learningRate * deltaJ[j] * inputWithBias[i]
                self.inputToHidden[i][j] += deltaW 
                #print "I->H weight delta", i, j, deltaW


In [3]:
import random
import numpy as np

#Clasificador de imagenes, las imagenes introducidas son de numeros del 0 al 9 escrtas a mano,
#cada renglon es una imagende un nuemro de 16x16
#los primeros 161 elemtos pertenen a ceros 
# de 161-323 pertenece a unos
# de 323-482 pertenece a dos 
# de 482-641 pertenece a tres 
# de 641-802 pertenece a cuatro
# de 802-961 pertenece a cinco 
# de 961-1122 pertenece a seis
# de 1122-1280 pertenece a siete 
# de 1280-1435 pertenece a ocho
# de 1435-1593 pertenece a nueve

#Obtencion de los datos
datos = np.genfromtxt("semeionP.data",dtype="int",delimiter=',')

#Separar los datos en cada una de sus clases.
cero = datos[:161]
uno = datos[161:323]
dos = datos[323:482]
tres = datos[482:641]
cuatro = datos[641:802]
cinco = datos[802:961]
seis = datos[961:1122]
siete = datos[1122:1280]
ocho = datos[1280:1435]
nueve = datos[1435:]

#Se crea una red neuronal de 256 entradas, 10 capas ocultas, 4 salidas, desviacion estandar de 0.5 y velocidad de aprendisaje de 0.8
red=NN(256,10,4,0.5,0.8)

entrenamientos = 150

#Se entrena 150 a la red neuronal.
for i in range(entrenamientos):
    # Se muestran solo 110 elemetos de cada conjunto.
    for j in range(110):
        salida,activacionesOcultas = red.calcNetOutput( list(cero[j]),True)
        red.trainingEpisode( [0,0,0,0], salida, activacionesOcultas,list(cero[j]))
        
        salida,activacionesOcultas = red.calcNetOutput( list(uno[j]),True)
        red.trainingEpisode( [0,0,0,1], salida, activacionesOcultas,list(uno[j]))
        
        salida,activacionesOcultas = red.calcNetOutput( list(dos[j]),True)
        red.trainingEpisode( [0,0,1,0], salida, activacionesOcultas,list(dos[j]))
        
        salida,activacionesOcultas = red.calcNetOutput( list(tres[j]),True)
        red.trainingEpisode( [0,0,1,1], salida, activacionesOcultas,list(tres[j]))
        
        salida,activacionesOcultas = red.calcNetOutput( list(cuatro[j]),True)
        red.trainingEpisode( [0,1,0,0], salida, activacionesOcultas,list(cuatro[j]))
        
        salida,activacionesOcultas = red.calcNetOutput( list(cinco[j]),True)
        red.trainingEpisode( [0,1,0,1], salida, activacionesOcultas,list(cinco[j]))
        
        salida,activacionesOcultas = red.calcNetOutput( list(seis[j]),True)
        red.trainingEpisode( [0,1,1,0], salida, activacionesOcultas,list(seis[j]))
        
        salida,activacionesOcultas = red.calcNetOutput( list(siete[j]),True)
        red.trainingEpisode( [0,1,1,1], salida, activacionesOcultas,list(siete[j]))
        
        salida,activacionesOcultas = red.calcNetOutput( list(ocho[j]),True)
        red.trainingEpisode( [1,0,0,0], salida, activacionesOcultas,list(ocho[j]))
        
        salida,activacionesOcultas = red.calcNetOutput( list(nueve[j]),True)
        red.trainingEpisode( [1,0,0,1], salida, activacionesOcultas,list(nueve[j]))
    #Muestra enque ciclo se encuentra    
    print("Ciclo: ",i)
print("Fin entrenamiento.")

#Se declaran contadores para contar el numero de aciertos por cada clase.
nCero = 0
nUno = 0
nDos = 0
nTres = 0
nCuatro = 0
nCinco = 0
nSeis = 0
nSiete = 0
nOcho = 0
nNueve = 0

#Se cuentan los aciertos de la red, con las clases que tiene 161 elemtos.
for i in range(len(cero)):
    salida = red.calcNetOutput(list(cero[i]))
    #El cero se reprenta como 0000
    if salida[0]<0.2 and salida[1]<0.2 and salida[2]<0.2 and salida[3]<0.2:
        nCero+=1
        
    salida = red.calcNetOutput(list(cuatro[i]))
    #El cuatro se representa como 0100
    if salida[0]<0.2 and salida[1]>0.8 and salida[2]<0.2 and salida[3]<0.2:
        nCuatro+=1
        
    salida = red.calcNetOutput(list(seis[i]))
    #El seis se representa como 0110
    if salida[0]<0.2 and salida[1]>0.8 and salida[2]>0.8 and salida[3]<0.2:
        nSeis+=1

#Se cuentan los aciertos de la red, con las clases que tienen 159 elemtos.
for i in range(len(dos)):
    salida = red.calcNetOutput(list(dos[i]))
    #El dos se representa como 0010
    if salida[0]<0.2 and salida[1]<0.2 and salida[2]>0.8 and salida[3]<0.2:
        nDos+=1
        
    salida = red.calcNetOutput(list(tres[i]))
    #El tres se representa como 0011
    if salida[0]<0.2 and salida[1]<0.2 and salida[2]>0.8 and salida[3]>0.8:
        nTres+=1
        
    salida = red.calcNetOutput(list(cinco[i]))
    #El cinco se representa como 0101
    if salida[0]<0.2 and salida[1]>0.8 and salida[2]<0.2 and salida[3]>0.8:
        nCinco+=1
        
#Se cuentan los aciertos de la red, con las clases que tienen 158 elemtos.
for i in range(len(siete)):    
    salida = red.calcNetOutput(list(siete[i]))
    #El numero siete se representa como 0111
    if salida[0]<0.2 and salida[1]>0.8 and salida[2]>0.8 and salida[3]>0.8:
        nSiete+=1
    
    salida = red.calcNetOutput(list(nueve[i]))
    #El numero nueve se representa como 1001
    if salida[0]>0.8 and salida[1]<0.2 and salida[2]<0.2 and salida[3]>0.8:
        nNueve+=1
        
#Se cuentan los aciertos de la red, con las clases que tienen 162 elemtos.
for i in range(len(uno)):
    salida = red.calcNetOutput(list(uno[i]))
    #El numero uno se representa como 0001
    if salida[0]<0.2 and salida[1]<0.2 and salida[2]<0.2 and salida[3]>0.8:
        nUno+=1

#Se cuentan los aciertos de la red, con las clases que tienen 155 elemtos.
for i in range(len(ocho)):
    salida = red.calcNetOutput(list(ocho[i]))
    if salida[0]>0.8 and salida[1]<0.2 and salida[2]<0.2 and salida[3]<0.2:
        nOcho+=1

#Muestra el porcentaje de acierto de la red y el porcentaje de acierto por cada clase.
print(f"El porcentaje de acierto de la red es: {(nCero+nUno+nDos+nTres+nCuatro+nCinco+nSeis+nSiete+nOcho+nNueve)/1593*100} %")

print(f"El porcentaje de Cero es: {nCero/161*100} %")

print(f"El porcentaje de Uno es: {nUno/162*100} %")

print(f"El porcentaje de Dos es: {nDos/159*100} %")

print(f"El porcentaje de Tres es: {nTres/159*100} %")

print(f"El porcentaje de Cuatro es: {nCuatro/161*100} %")

print(f"El porcentaje de Cinco es: {nCinco/159*100} %")

print(f"El porcentaje de Seis es: {nSeis/161*100} %")

print(f"El porcentaje de Siete es: {nSiete/158*100} %")

print(f"El porcentaje de Ocho es: {nOcho/155*100} %")

print(f"El porcentaje de Nueve es: {nNueve/158*100} %")


#Muestra la imagen de un numero, se tiene que ingresar un indice en datos[indice]
# numPixel=0
# for pixel in datos[1592]:
#     if pixel == 1:
#         print("▓▓",end="")
#         numPixel += 1
#     else:
#         print("░░",end="")
#         numPixel+=1
#     if numPixel == 16:
#         print()
#         numPixel = 0


#Muestra la imagen de los numeros en un rango
# range(161) para mostrar los ceros
# range(161,323) para mostrar los unos
# range(323,482) para mostrar los dos 
# range(482,641) para mostrar los tres 
# range(641,802) para mostrar los cutro
# range(802,961) para mostrar los cinco 
# range(961,1122) para mostrar los seis
# range(1122,1280) para mostrar los siete 
# range(1280,1435) para mostrar los ocho
# range(1435,1593) para mostrar los nueve

# numImagenes = 0
# for i in range(483,642):
#     numPixel=0
#     for pixel in datos[i]:
#         if pixel == 1:
#             print("▓▓",end="")
#             numPixel += 1
#         else:
#             print("░░",end="")
#             numPixel+=1
#         if numPixel == 16:
#             print()
#             numPixel = 0
#     print()
    #Para mostar solo una porcion de imagenes, reemplazar 1 con el numero de elemtos que desea mostar.
    # numImagenes += 1
    # if numImagenes == 5:
    #     break

Ciclo:  0
Ciclo:  1
Ciclo:  2
Ciclo:  3
Ciclo:  4
Ciclo:  5
Ciclo:  6
Ciclo:  7
Ciclo:  8
Ciclo:  9
Ciclo:  10
Ciclo:  11
Ciclo:  12
Ciclo:  13
Ciclo:  14
Ciclo:  15
Ciclo:  16
Ciclo:  17
Ciclo:  18
Ciclo:  19
Ciclo:  20
Ciclo:  21
Ciclo:  22
Ciclo:  23
Ciclo:  24
Ciclo:  25
Ciclo:  26
Ciclo:  27
Ciclo:  28
Ciclo:  29
Ciclo:  30
Ciclo:  31
Ciclo:  32
Ciclo:  33
Ciclo:  34
Ciclo:  35
Ciclo:  36
Ciclo:  37
Ciclo:  38
Ciclo:  39
Ciclo:  40
Ciclo:  41
Ciclo:  42
Ciclo:  43
Ciclo:  44
Ciclo:  45
Ciclo:  46
Ciclo:  47
Ciclo:  48
Ciclo:  49
Ciclo:  50
Ciclo:  51
Ciclo:  52
Ciclo:  53
Ciclo:  54
Ciclo:  55
Ciclo:  56
Ciclo:  57
Ciclo:  58
Ciclo:  59
Ciclo:  60
Ciclo:  61
Ciclo:  62
Ciclo:  63
Ciclo:  64
Ciclo:  65
Ciclo:  66
Ciclo:  67
Ciclo:  68
Ciclo:  69
Ciclo:  70
Ciclo:  71
Ciclo:  72
Ciclo:  73
Ciclo:  74
Ciclo:  75
Ciclo:  76
Ciclo:  77
Ciclo:  78
Ciclo:  79
Ciclo:  80
Ciclo:  81
Ciclo:  82
Ciclo:  83
Ciclo:  84
Ciclo:  85
Ciclo:  86
Ciclo:  87
Ciclo:  88
Ciclo:  89
Ciclo:  90
Ciclo:  9